In [2]:
import os
import re
import requests
import numpy as np
from xml.etree import ElementTree
from google.cloud import bigquery

# ----------------------------
# LawLoader クラス（上記のクラス定義）
# ----------------------------
class LawLoader(object):
    """
    Prepare law data with e-Gov (https://www.e-gov.go.jp/) site.

    Args:
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)
    """
    def __init__(self, category=1):
        self.law_dict = self._get_law_dict(category=category)
        self.content_dict = {}

    @staticmethod
    def _get_xml(url):
        """
        Get XML data from e-Gov API.

        Args:
            url (str): key of the API

        Returns:
            xml.ElementTree: element tree of the XML data
        """
        r = requests.get(url)
        return ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    def _get_law_dict(self, category):
        """
        Return dictionary of law names and numbers.

        Args:
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law names (keys) and numbers (values)
        """
        url = f"https://elaws.e-gov.go.jp/api/1/lawlists/{category}"
        root = self._get_xml(url)
        names = [e.text for e in root.iter() if e.tag == "LawName"]
        numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
        return {name: num for (name, num) in zip(names, numbers)}

    def get_law_number(self, keyword, category=1):
        """
        Return the law number.
        This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

        Args:
            keyword (str): keyword of the law name
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law name (key) and law number (value)
        """
        return {k: v for (k, v) in self.law_dict.items() if keyword in k}

    def get_raw(self, number):
        """
        Args:
            number (str): Number of the law, like '平成九年厚生省令第二十八号'

        Returns:
            raw (list[str]): raw contents of J-GCP
        """
        if number in self.content_dict:
            return self.content_dict[number]
        url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
        root = self._get_xml(url)
        contents = [e.text.strip() for e in root.iter() if e.text]
        raw = [t for t in contents if t]
        self.content_dict[number] = raw
        return raw

    @staticmethod
    def pre_process(raw):
        """
        Perform pre-processing on raw contents.

        Args:
            raw (list[str]): raw contents

        Returns:
            str: pre-processed string

        Notes:
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        contents = [s for s in raw if s.endswith("。")]
        string = "".join(contents)
        string = string.translate(str.maketrans({"「": "", "」": ""}))
        return re.sub("（[^（|^）]*）", "", string)

    def gcp(self):
        """
        Perform pre-processing on raw contents of J-GCP.

        Returns:
            str: pre-processed string of J-GCP

        Notes:
            - Article 56 will be removed.
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        number_dict = self.get_law_number("医薬品の臨床試験")
        number = number_dict["医薬品の臨床試験の実施の基準に関する省令"]
        raw = self.get_raw(number)
        raw_without56 = raw[: raw.index("第五十六条")]
        return self.pre_process(raw_without56)

# ----------------------------
# 埋め込み生成の関数（ここではダミー例）
# ----------------------------
def compute_embedding(text: str) -> np.ndarray:
    """
    与えられたテキストから埋め込みベクトルを生成する関数。
    ※実際にはVertex AI の埋め込みモデルなどを利用してください。
    """
    # 例としてテキストの長さに基づいて乱数シードを設定し、768次元の乱数ベクトルを返す
    np.random.seed(len(text))
    return np.random.rand(768)

# ----------------------------
# BigQuery にデータを挿入する関数
# ----------------------------
def insert_law_into_bigquery(law_data: dict, embedding: np.ndarray) -> None:
    """
    取得した法令データとその埋め込みベクトルをBigQueryのテーブルに挿入する関数。

    Args:
        law_data (dict): 法令の情報（例：lawId, lawName, lawText）
        embedding (np.ndarray): 埋め込みベクトル
    """
    # BigQuery クライアントの初期化（環境変数 GOOGLE_PROJECT_ID が設定済みの場合）
    client = bigquery.Client(project="zennaihackason")
    
    table_id = "zennaihackason.law_db.law_embeddings"
    
    # 挿入する行の整形（埋め込みはリスト形式に変換）
    row = {
        "law_id": law_data.get("lawId", ""),
        "law_name": law_data.get("lawName", ""),
        "law_text": law_data.get("lawText", ""),
        "embedding": embedding.tolist(),
    }
    
    errors = client.insert_rows_json(table_id, [row])
    if errors:
        print("BigQuery 挿入時のエラー:", errors)
    else:
        print("BigQuery へのデータ挿入に成功しました。")

# ----------------------------
# LawLoader を用いたデータ取得から BigQuery 挿入までの流れ
# ----------------------------
def main():
    # 例として category 2（法令）のデータを扱う
    loader = LawLoader(category=1)
    
    # キーワードで対象法令の番号を取得
    law_numbers = loader.get_law_number("医薬品の臨床試験")
    print("対象法令の一覧:", law_numbers)
    target_law_name = "医薬品の臨床試験の実施の基準に関する省令"
    if target_law_name in law_numbers:
        law_number = law_numbers[target_law_name]
        print(f"対象法令番号: {law_number}")
        
        # 法令の生テキストを取得し、前処理を実施
        raw_data = loader.get_raw(law_number)
        preprocessed_text = loader.pre_process(raw_data)
        
        # 埋め込み生成（実際のモデルに置き換え可能）
        embedding_vector = compute_embedding(preprocessed_text)
        
        # BigQuery に挿入するためのレコード作成
        law_record = {
            "lawId": law_number,
            "lawName": target_law_name,
            "lawText": preprocessed_text,
        }
        
        # BigQuery にデータを挿入
        insert_law_into_bigquery(law_record, embedding_vector)
    else:
        print("対象の法令が見つかりませんでした。")

main()


対象法令の一覧: {'医薬品の臨床試験の実施の基準に関する省令': '平成九年厚生省令第二十八号', '動物用医薬品の臨床試験の実施の基準に関する省令': '平成九年農林水産省令第七十五号'}
対象法令番号: 平成九年厚生省令第二十八号
BigQuery へのデータ挿入に成功しました。
